### wps_climdex_ptot

WPS wrapper for [climdex.pcic](https://cran.r-project.org/web/packages/climdex.pcic/climdex.pcic.pdf) functions that calculate annual timeseries of precipitation exceeding the threshold 

- **PRCpTOT** ([climdex.r95ptot](https://cran.r-project.org/web/packages/climdex.pcic/climdex.pcic.pdf#page=14)) The annual sum of precipitation in wet days (days where precipitation is at least 1mm).
- **R95pTOT** ([climdex.r95ptot](https://cran.r-project.org/web/packages/climdex.pcic/climdex.pcic.pdf#page=18)) The annual sum of precipitation in days where daily precipitation exceeds the 95th percentile of daily precipitation in the base period.
- **R99pTOT** ([climdex.r99ptot](https://cran.r-project.org/web/packages/climdex.pcic/climdex.pcic.pdf#page=19)) The annual sum of precipitation in days where daily precipitation exceeds the 99th percentile of daily precipitation in the base period.

In [2]:
import os
import requests
from birdy import WPSClient
from rpy2 import robjects
from urllib.request import urlretrieve
from pkg_resources import resource_filename
from tempfile import NamedTemporaryFile

from wps_tools.R import rda_to_vector, construct_r_out, test_rda_output
from wps_tools.testing import get_target_url

In [3]:
# Ensure we are in the working directory with access to the data
while os.path.basename(os.getcwd()) != "quail":
    os.chdir('../')

In [4]:
# NBVAL_IGNORE_OUTPUT
url = get_target_url("quail")
print(f"Using quail on {url}")

Using quail on https://docker-dev03.pcic.uvic.ca/twitcher/ows/proxy/quail/wps


In [5]:
quail = WPSClient(url)

#### Help for individual processes can be diplayed using the ? command (ex/ bird.process?)

In [6]:
# NBVAL_IGNORE_OUTPUT
quail.climdex_ptot?

Signature:
quail.climdex_ptot(
    climdex_input=None,
    loglevel='INFO',
    output_file='output.rda',
    threshold=0,
    output_formats=None,
)
Docstring:
Total daily precipitation exceeding threshold

Parameters
----------
climdex_input : ComplexData:mimetype:`application/x-gzip`
    RDS or Rdata (.rds, .rda, .rdata) file containing R Object of type climdexInput
output_file : string
    Filename to store the output Rdata (extension .rda)
threshold : {'0', '95', '99'}integer
    Daily precipitation threshold
loglevel : {'CRITICAL', 'ERROR', 'WARNING', 'INFO', 'DEBUG', 'NOTSET'}string
    Logging level

Returns
-------
rda_output : ComplexData:mimetype:`application/x-gzip`
    Rda file containing R output data
File:      ~/github/pcic/quail/</tmp/quail-venv/lib/python3.8/site-packages/birdy/client/base.py-10>
Type:      method


### Run wps_climdex_ptot Process for climdex.prcptot()

In [7]:
with NamedTemporaryFile(suffix=".rda", prefix="prcptot_", dir="/tmp", delete=True) as output_file:
    output = quail.climdex_ptot(
            climdex_input=resource_filename("tests","data/climdexInput.rda"),
            output_file=output_file.name,
        )
prcptot_url = output.get()[0]

You can also use rds input
### Run wps_climdex_ptot Process for climdex.r99ptot() with rds input

In [8]:
with NamedTemporaryFile(suffix=".rda", prefix="r99ptot_", dir="/tmp", delete=True) as output_file:
    output = quail.climdex_ptot(
            climdex_input=resource_filename("tests","data/climdexInput.rds"),
            threshold=99,
            output_file=output_file.name,
        )
r99ptot_url = output.get()[0]

You can use multiple input files
### Run wps_climdex_ptot Process for climdex.r95ptot() with multiple input files

In [9]:
climdex_inputs = [
    resource_filename("tests","data/climdexInput.rds"),
    resource_filename("tests","data/climdexInput.rda"),
    resource_filename("tests","data/climdex_input_multiple.rda")
]
with NamedTemporaryFile(suffix=".rda", prefix="r95ptot_", dir="/tmp", delete=True) as output_file:
    output = quail.climdex_ptot(
            climdex_input=climdex_inputs,
            output_file=output_file.name,
            threshold=95,
        )
r95ptot_url = output.get()[0]

Access the output with **rda_to_vector** or **construct_r_out** from **wps_tools.R**

In [10]:
# NBVAL_IGNORE_OUTPUT
# use print() to see whole vector
prcptot = rda_to_vector(prcptot_url, "prc0_ci")
print(f"prcptot\n{prcptot}")
r95ptot = rda_to_vector(r95ptot_url, "r950_ci")
print(f"r95ptot\n{r95ptot}")
r99ptot = rda_to_vector(r99ptot_url, "r990_ci")
print(f"r99ptot\n{r99ptot}")

prcptot
  1959   1960   1961   1962   1963   1964   1965   1966   1967   1968   1969 
    NA  803.7     NA     NA     NA  877.6  874.3  771.1 1080.6 1029.5  652.0 
  1970   1971   1972   1973   1974   1975   1976   1977   1978   1979   1980 
 771.6 1030.8 1097.8  670.1  863.6 1000.1     NA     NA  585.4  715.7  991.4 
  1981   1982   1983   1984   1985   1986   1987   1988   1989   1990   1991 
 907.8     NA  913.2     NA     NA     NA     NA     NA  860.4 1292.5     NA 
  1992   1993   1994   1995   1996   1997   1998   1999   2000   2001   2002 
 797.2     NA     NA 1182.1     NA 1261.3 1093.5 1266.6  648.5  921.9     NA 
  2003   2004 
1105.5     NA 

r95ptot
 1959  1960  1961  1962  1963  1964  1965  1966  1967  1968  1969  1970  1971 
   NA 226.6    NA    NA    NA 140.2 256.3 138.4 349.3 165.3  70.7  50.8 261.6 
 1972  1973  1974  1975  1976  1977  1978  1979  1980  1981  1982  1983  1984 
390.9  80.8 186.4 295.2    NA    NA  62.0 195.4 254.2  93.0    NA 171.4    NA 
 1985  1986  

In [11]:
# NBVAL_IGNORE_OUTPUT
construct_r_out([prcptot_url, r95ptot_url, r99ptot_url])

[[R object with classes: ('numeric',) mapped to:
  [     nan, 803.699996,      nan,      nan, ..., 921.899999,      nan, 1105.500000,      nan]],
 [R object with classes: ('numeric',) mapped to:
  [     nan, 226.599997,      nan,      nan, ..., 155.400000,      nan, 335.500000,      nan],
  R object with classes: ('numeric',) mapped to:
  [     nan, 226.599997,      nan,      nan, ..., 155.400000,      nan, 335.500000,      nan],
  R object with classes: ('numeric',) mapped to:
  [     nan, 226.599997,      nan,      nan, ..., 155.400000,      nan, 335.500000,      nan],
  R object with classes: ('numeric',) mapped to:
  [     nan, 181.899997,      nan,      nan, ..., 155.400000,      nan, 312.500000,      nan]],
 [R object with classes: ('numeric',) mapped to:
  [     nan, 152.399997,      nan,      nan, ..., 0.000000,      nan, 169.000000,      nan]]]

#### Test output against expected output

In [12]:
test_rda_output(
    prcptot_url, "prc0_ci", "expected_ptot.rda", "expected_prcptot"
    )

test_rda_output(
    r95ptot_url, "r950_ci", "expected_ptot.rda", "expected_r95ptot"
    )

test_rda_output(
    r99ptot_url, "r990_ci", "expected_ptot.rda", "expected_r99ptot"
    )